Уходим от списков к массивам

In [366]:
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import random 
#!pip install import-ipynb
#import import_ipynb
import modelparameters as mpar
import numpy as np
from sympy import *
%load_ext Cython
import time
import array

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [372]:
%%cython 
cimport numpy as np
from numpy cimport int32_t
from cython.view cimport array as carray
cdef extern from "math.h":
    double sin(double)
    const double pi "M_PI"
cdef float aep = 1.1560
cdef float app = 0.8669
cdef float m_p = 1836.2
cdef float E_0 = 0.6
cdef float w_0 = 0.058
cdef int N = 3
cdef float T = 2 * pi * N /w_0

cpdef  double length2(double[:] R,  double[:] r):
    return pow((R[0] - r[0]),2)  + pow((R[1] - r[1]),2) + pow((R[2] - r[2]),2)
cpdef  float f_e(int i,  double[:] R_1,  double[:] R_2,  double[:] r):
    return (R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)), (3 / 2))) + (
                (R_2[i] - r[i])) /  (
                pow((length2(R_2,r) + pow(aep, 2)),(3 / 2)))
cpdef float f_p1(int i,  double[:] R_1,  double[:] R_2,  double[:] r) :
        return (-(R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)),  (3 / 2))) + (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))

cpdef float f_p2(int i,  double[:] R_1,  double[:] R_2,  double[:] r) :
        return (-(R_2[i] - r[i]) / (
                pow((length2(R_2,r) + pow(aep,2)) , (3 / 2))) - (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))
cpdef  dif( double[:]a, t) :
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    cdef double [3] electron = [x, y, z]
    cdef double [3] proton1 = [x_1, y_1, z_1]
    cdef double [3] proton2 = [x_2, y_2, z_2]
    dadt = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0,proton1,proton2, electron),
            f_e(1,proton1,proton2, electron),
            f_e(2, proton1,proton2, electron),
            f_p1(0, proton1,proton2, electron),
            f_p1(1, proton1,proton2, electron),
            f_p1(2, proton1,proton2, electron),
            f_p2(0, proton1,proton2, electron),
            f_p2(1, proton1,proton2, electron),
            f_p2(2, proton1,proton2, electron)]
    
    return dadt
cpdef float E(t):
    E = E_0 * pow((sin(pi * t / T)),2) * sin(w_0 * t)
    return E
cpdef  dif2(double[:]a, t2):
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    cdef int L = 100000
    cdef double [3] electron = [x, y, z]
    cdef double [3] proton1 = [x_1, y_1, z_1]
    cdef double [3] proton2 = [x_2, y_2, z_2]
    dadt  = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0, proton1,proton2, electron),
            f_e(1, proton1,proton2, electron),
            f_e(2, proton1,proton2, electron)- E(t2),
            f_p1(0,  proton1,proton2, electron),
            f_p1(1,  proton1,proton2, electron),
            f_p1(2,  proton1,proton2, electron) + E(t2), 
            f_p2(0,  proton1,proton2, electron),
            f_p2(1,  proton1,proton2, electron),
            f_p2(2,  proton1,proton2, electron) + E(t2)] 
    return dadt

In [309]:
p = mpar.p
t = np.linspace(0, 50, 100)
#def a_0(i):
a_0 =  np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 #   return a_0
#print(dif2,'uyi',a_0,'niu', t) 
(odeint(dif, a_0, t))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [369]:
p = mpar.p
R_1 = np.array([-mpar.dist/2, 0.0, 0.0]) #начальные положения pr1,pr2,e
R_2 = np.array([mpar.dist/2, 0.0, 0.0])
r = np.array([0,0,0])

print (mpar.dist)
def a_0(i):
    a_0 = np.array([0.0, 0.0, 0.0,
                    -mpar.dist / 2, 0.0, 0.0,
                    mpar.dist / 2, 0.0, 0.0,
                    p[i][0], p[i][1], p[i][2], 
                    0.0, 0.0, 0.0, 
                    0.0, 0.0, 0.0])
    return a_0
solution =  []
start = time.time()
odeint(dif, a_0(i), mpar.t)
#Решения 18 уравнений для n экспериментов
for i in range (mpar.n):
    solution.append(odeint(dif, a_0(i), mpar.t))
#print ("rfn", solution[1][mpar.L -1,3])


#Решаем сиситему 18 диф. уравнений с лазером
t2 = np.linspace(mpar.u, mpar.u + 325, 100000)

def a_00(i):
    a_00 = solution[i][mpar.L -1]
    return a_00
solution2 = []


# Решения 18 уравнений для n экспериментов
for i in range(mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))

end = time.time()
vt = end-start
print('Время выполнения:',vt)
print('Среднее время выполнения одного эксперимента:',vt/mpar.nn)

1.5070016255625132
Время выполнения: 20.495975971221924
Среднее время выполнения одного эксперимента: 2.0495975971221925


In [345]:
first = time.time()
solution2 = []
t2 = linspace(mpar.u, mpar.u + 325, 100000)
def a_00(i):
    a_00 = solution[i][mpar.L -1]
    return a_00
for i in range (mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))
firstend = time.time()
print(firstend-first)

1.8179149627685547


In [347]:
first = time.time()
solution2 = []
t2 = linspace(mpar.u, mpar.u + 325, 100000)
def a_00(i):
    a_00 = np.array([solution[i][mpar.L - 1, 0], solution[i][mpar.L - 1, 1], solution[i][mpar.L - 1, 2], 
                     solution[i][mpar.L - 1, 3], solution[i][mpar.L - 1, 4], solution[i][mpar.L - 1, 5],
                     solution[i][mpar.L - 1, 6], solution[i][mpar.L - 1, 7], solution[i][mpar.L - 1, 8],
                     solution[i][mpar.L - 1, 9], solution[i][mpar.L - 1, 10], solution[i][mpar.L - 1, 11],
                     solution[i][mpar.L - 1, 12], solution[i][mpar.L - 1, 13], solution[i][mpar.L - 1, 14],
                     solution[i][mpar.L - 1, 15], solution[i][mpar.L - 1, 16], solution[i][mpar.L - 1, 17]])
    return a_00
for i in range (mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))
firstend = time.time()
print(firstend-first)

1.7805798053741455
